## Разбор моделей турниров 

Подготовил Григорий Крюков

Рассмотрим две модели учета психологических эффектов в турнирах.

В обеих моделях предполагаем, что турнир проходит по круговой системе, в матчах возможны лишь два результата: победа и поражение. Для каждого из игроков задана его сила $w_i$ и матрица переходов $A^i$ размера $2 \times 2$. Матрицы $A^i$ неотрицательные, сумма элементов каждой из двух строк равна единице. Дополнительно предполагаем, что $A_{11}^i > A_{21}^i$. Идея в том, что после победы легче победить еще раз, чем после поражения.

$A_{11}^i$ - вероятность победы $i$-го игрока после победы в прошлой игре.

$A_{12}^i$ - вероятность поражения $i$-го игрока после победы в прошлой игре.

$A_{21}^i$ - вероятность победы $i$-го игрока после поражения в прошлой игре.

$A_{22}^i$ - вероятность поражения $i$-го игрока после поражения в прошлой игре.

В каждой из двух моделей с некоторой вероятностью наступает победа "на классе". Пусть победа $i$-го игрока над $j$-ым на классе задается функцией $f(w_i, w_j)$, которая принимает значения от $0$ до $1$.

В каждой модели дополнительно предполагаем, что у всех игроков эмоциональное состояние идентично состоянию при победе в прошлой игре.

#### Первая модель.

Вероятность победы $i$-ой команды на морально-волевых качествах: $A_{k1}^i A_{l2}^j$

Вероятность победы $i$-ой команды на морально-волевых качествах: $A_{k2}^i A_{l1}^j$

Где $k$ - прошлый результат $i$-ой команды, $l$ - прошлый результат $j$-ой команды.

С вероятностью $1 - A_{k1}^i A_{l2}^j - A_{k2}^i A_{l1}^j$ результат определяется по классу команд, вероятность задается функцией $f$.

Итоговая вероятность победы $i$-ой команды над $j$-ой:

$$P_i = A_{k1}^i A_{l2}^j + (1 - A_{k1}^i A_{l2}^j - A_{k2}^i A_{l1}^j) f(w_i, w_j)$$

#### Вторая модель.

Без ограничения общности положим, что $w_i \geq w_j$.

Тогда считаем, что $i$-ый игрок побеждает на классе с вероятностью $f(w_i, w_j)$. В остальных случаях победа определяется по морально-волевым качествам.

Вероятность победы $i$-ой команды на морально-волевых качествах: $(1 - f(w_i, w_j))\frac{A_{k1}^i A_{l2}^j}{A_{k1}^i A_{l2}^j + A_{k2}^i A_{l1}^j}$

Вероятность победы $j$-ой команды на морально-волевых качествах: $(1 - f(w_i, w_j))\frac{A_{k2}^i A_{l1}^j}{A_{k1}^i A_{l2}^j + A_{k2}^i A_{l1}^j}$

Итого вероятность победы $i$-ой команды: 

$$P_i = f(w_i, w_j) + (1 - f(w_i, w_j))\frac{A_{k1}^i A_{l2}^j}{A_{k1}^i A_{l2}^j + A_{k2}^i A_{l1}^j}$$

Где $w_i \geq w_j$, $k$ - прошлый результат $i$-ой команды, $l$ - прошлый результат $j$-ой команды.

## I. Синтетические эксперименты

In [17]:
import numpy as np
import itertools
from scipy import stats
from scipy.stats import bernoulli
from tqdm import tqdm

""" get_array_of_powers() генерирует отсортированный по возрастанию массив из N сил игроков,
который генерируется из распределения distrib"""
def get_array_of_powers(N = 4, distrib = stats.uniform):
    arr = distrib.rvs(size = N)
    arr.sort()
    return arr

""" get_list_of_transformation_matrices() генерирует N матриц перехода,
первый столбец каждой из которых генерируется из распределения distrib"""
def get_list_of_transformation_matrices(N = 4, distrib = stats.uniform, sort = True):
    output = []
    for _ in range(N):
        prob = distrib.rvs(size = 2)
        if sort:
            prob[::-1].sort() # Сортировка по убыванию: вероятность победы выше, если прошлый результат был победой
        arr = np.vstack((prob, 1 - prob)).T # Объединяем столбцы матрицы переходов
        output.append(arr)
    return output

""" generate_all_schedules_4_teams() генерирует все возможные расписания для турнира на 4 команды.
Метод основан на том, что соперник первой команды однозначно определяет все пары на тур.
Соответственно, есть 3! возможных расписаний"""
def generate_all_schedules_4_teams():
    N = 4
    output = []
    teams = [i for i in range(N)]
    for perm in itertools.permutations(teams[1:], 3):
        schedule = []
        for i in perm:
            round_ = []
            round_.append([0, i])
            round_.append(list(set(teams[1:]).difference(set([i]))))
            schedule.append(round_)
        output.append(schedule)
    return output

""" start_tournament_FM() проводит турнир в соответствии с первой моделью.
powers - массив сил игроков
A - список матриц переходов
schedule - расписание турнира
N - число игроков
f - функция из модели
Возвращает число очков, которые набрала каждая из команд"""
def start_tournament_FM(powers, A, schedule, N = 4, f = stats.norm.cdf):
    points = [0 for _ in range(N)]
    moods = [0 for _ in range(N)] # 0 - прошлая победа, 1 - прошлое поражение
    for rnd in schedule:
        for game in rnd:
            j = game[0]
            i = game[1]
            P = A[i][moods[i]][0] * A[j][moods[j]][1] + (1 - A[i][moods[i]][0] * A[j][moods[j]][1] - A[i][moods[i]][1] * A[j][moods[j]][0]) * f(powers[i] - powers[j])
            win_i = bernoulli(P).rvs()
            points[i] += win_i
            points[j] += 1 - win_i
            moods[i] = 1 - win_i
            moods[j] = win_i
    return points

""" start_tournament_SM() проводит турнир в соответствии со второй моделью.
powers - массив сил игроков
A - список матриц переходов
schedule - расписание турнира
N - число игроков
f - функция из модели
Возвращает число очков, которые набрала каждая из команд"""
def start_tournament_SM(powers, A, schedule, N = 4, f = stats.norm.cdf):
    points = [0 for _ in range(N)]
    moods = [0 for _ in range(N)] # 0 - прошлая победа, 1 - прошлое поражение
    for rnd in schedule:
        for game in rnd:
            j = game[0]
            i = game[1]
            P = f(powers[i] - powers[j]) + (1 - f(powers[i] - powers[j])) * A[i][moods[i]][0] * A[j][moods[j]][1] / (A[i][moods[i]][0] * A[j][moods[j]][1] + A[i][moods[i]][1] * A[j][moods[j]][0])
            win_i = bernoulli(P).rvs()
            points[i] += win_i
            points[j] += 1 - win_i
            moods[i] = 1 - win_i
            moods[j] = win_i
    return points

Продемонстрируем работу функций.

In [18]:
get_array_of_powers()

array([0.06709373, 0.59370578, 0.77203179, 0.82821276])

In [19]:
get_array_of_powers(N=7, distrib = stats.norm)

array([-1.71060335, -1.56798827, -1.02792377, -0.3930902 ,  0.33191845,
        0.81170557,  2.93540621])

In [20]:
get_list_of_transformation_matrices()

[array([[0.28686687, 0.71313313],
        [0.2685796 , 0.7314204 ]]),
 array([[0.86106294, 0.13893706],
        [0.06986192, 0.93013808]]),
 array([[0.67761118, 0.32238882],
        [0.17282005, 0.82717995]]),
 array([[0.34450589, 0.65549411],
        [0.06154303, 0.93845697]])]

In [21]:
generate_all_schedules_4_teams()

[[[[0, 1], [2, 3]], [[0, 2], [1, 3]], [[0, 3], [1, 2]]],
 [[[0, 1], [2, 3]], [[0, 3], [1, 2]], [[0, 2], [1, 3]]],
 [[[0, 2], [1, 3]], [[0, 1], [2, 3]], [[0, 3], [1, 2]]],
 [[[0, 2], [1, 3]], [[0, 3], [1, 2]], [[0, 1], [2, 3]]],
 [[[0, 3], [1, 2]], [[0, 1], [2, 3]], [[0, 2], [1, 3]]],
 [[[0, 3], [1, 2]], [[0, 2], [1, 3]], [[0, 1], [2, 3]]]]

In [22]:
powers = get_array_of_powers()
A = get_list_of_transformation_matrices()
schedule = generate_all_schedules_4_teams()[0]
start_tournament_FM(powers, A, schedule)

[2, 3, 1, 0]

In [23]:
powers = get_array_of_powers()
A = get_list_of_transformation_matrices()
schedule = generate_all_schedules_4_teams()[0]
start_tournament_SM(powers, A, schedule)

[1, 2, 1, 2]

Теперь проведем эксперименты. Будем случайно генерировать силы игроков и их матицы перехода, после чего проводить турнир по каждому из возможных расписаний. 

В качестве метрики качества будем использовать вероятность победы сильнейшего игрока на турнире. 

Если несколько игроков набрали одинаковое число очков, то места распределяются по жребию, каждый из этих игроков занимает наивысшее место равновероятно.

In [25]:
np.random.seed(42) # Фиксируем seed для воспроизводимости

iters = 10000
schedules = generate_all_schedules_4_teams()
prob_best_win_FM = np.zeros(len(schedules))
prob_best_win_SM = np.zeros(len(schedules))

for _ in tqdm(range(iters)):
    powers = get_array_of_powers()
    A = get_list_of_transformation_matrices()
    for i in range(len(schedules)):
        points = start_tournament_FM(powers, A, schedules[i])
        if points[3] == 3:
            prob_best_win_FM[i] += 1
        elif points[3] == 2:
            if points[0] == 0 or points[1] == 0 or points[2] == 0:
                prob_best_win_FM[i] += 1/3
            else:
                prob_best_win_FM[i] += 0.5
        points = start_tournament_SM(powers, A, schedules[i])
        if points[3] == 3:
            prob_best_win_SM[i] += 1
        elif points[3] == 2:
            if points[0] == 0 or points[1] == 0 or points[2] == 0:
                prob_best_win_SM[i] += 1/3
            else:
                prob_best_win_SM[i] += 0.5

print(prob_best_win_FM / iters)
print(prob_best_win_SM / iters)

100%|██████████| 10000/10000 [07:34<00:00, 21.99it/s]

[0.39168333 0.39896667 0.3895     0.40783333 0.3895     0.39553333]
[0.72096667 0.7487     0.7228     0.73135    0.71465    0.7273    ]


In [26]:
# Теперь рассмотрим нормальное распределение для сил игроков

np.random.seed(42) # Фиксируем seed для воспроизводимости

iters = 10000
schedules = generate_all_schedules_4_teams()
prob_best_win_FM = np.zeros(len(schedules))
prob_best_win_SM = np.zeros(len(schedules))

for _ in tqdm(range(iters)):
    powers = get_array_of_powers(distrib = stats.norm)
    A = get_list_of_transformation_matrices()
    for i in range(len(schedules)):
        points = start_tournament_FM(powers, A, schedules[i])
        if points[3] == 3:
            prob_best_win_FM[i] += 1
        elif points[3] == 2:
            if points[0] == 0 or points[1] == 0 or points[2] == 0:
                prob_best_win_FM[i] += 1/3
            else:
                prob_best_win_FM[i] += 0.5
        points = start_tournament_SM(powers, A, schedules[i])
        if points[3] == 3:
            prob_best_win_SM[i] += 1
        elif points[3] == 2:
            if points[0] == 0 or points[1] == 0 or points[2] == 0:
                prob_best_win_SM[i] += 1/3
            else:
                prob_best_win_SM[i] += 0.5

print(prob_best_win_FM / iters)
print(prob_best_win_SM / iters)

100%|██████████| 10000/10000 [07:26<00:00, 22.42it/s]

[0.53408333 0.54715    0.53026667 0.54408333 0.5316     0.54305   ]
[0.86313333 0.87446667 0.86476667 0.86583333 0.85758333 0.87445   ]
